In [ ]:
! pip install datasets transformers rouge-score nltk SentencePiece
! pip install --upgrade pyyaml

     |████████████████████████████████| 264 kB 14.8 MB/s 
     |████████████████████████████████| 2.6 MB 55.2 MB/s 
     |████████████████████████████████| 1.2 MB 46.3 MB/s 
     |████████████████████████████████| 118 kB 72.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 243 kB 85.1 MB/s 
     |████████████████████████████████| 895 kB 72.8 MB/s 
     |████████████████████████████████| 3.3 MB 56.4 MB/s 
     |████████████████████████████████| 636 kB 61.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install hf-lfs
!git config --global user.email "jakerutherford@outlook.com"
!git config --global user.name "JakeMSc"

     |████████████████████████████████| 4.0 MB 16.8 MB/s 


In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: JakeMSc
Password: 
Login successful
Your token: hAZfREWUQijkQBHfmbnHEIPIwxTDREXbyBkcGueDblJagPmuXLRagIJaRUiHNaEcQhywCGvKExdpEQRVATeamWsVUOPitqJWcWZygoRNmGHzenCuNuzXqHCPwBjBIAid 

Your token has been saved to /root/.huggingface/token


In [ ]:
!transformers-cli repo create "pegasus-large-legal"

2021-08-14 01:38:14.686263: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.7/dist-packages/transformers/commands/user.py:269: UserWarning: Managing repositories through transformers-cli is deprecated. Please use `huggingface-cli` instead.
  "Managing repositories through transformers-cli is deprecated. Please use `huggingface-cli` instead."
git version 2.17.1
git-lfs/2.13.3 (GitHub; linux amd64; go 1.16.2; git a5e65851)

You are about to create JakeMSc/pegasus-large-legal
Proceed? [Y/n] n
Abort


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from datasets import load_dataset

dataset_file = 'drive/MyDrive/Colab_Notebooks/Pegasus_training_data_medium.csv'

dataset = load_dataset('csv', data_files=dataset_file)

Using custom data configuration default-d5dde15000e0fbfd


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d5dde15000e0fbfd/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [ ]:
  dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'input_text', 'target_text'],
        num_rows: 29988
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_checkpoint = "google/pegasus-large"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

my_model = 'JakeMSc/pegasus-large-legal'

#tokenizer = AutoTokenizer.from_pretrained(my_model)  
model = AutoModelForSeq2SeqLM.from_pretrained(my_model, use_auth_token=True).to(torch_device)

In [ ]:
max_input_length = 512
def preprocess_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=max_input_length, truncation=True, padding=True)

    labels = tokenizer(examples["target_text"], max_length=max_input_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/30 [00:00<?, ?ba/s]

In [ ]:
batch_size = 1
args = Seq2SeqTrainingArguments(
    output_dir='./results',
    #evaluation_strategy = "epoch",
    #predict_with_generate=True,
    do_train=True,
    learning_rate=0.01,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    logging_dir='./logs',
    logging_steps=100,
    adafactor=True,
    push_to_hub=True,
    push_to_hub_model_id=my_model,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

HTTPError: ignored

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: Unnamed: 0, input_text, target_text.
***** Running training *****
  Num examples = 29988
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 29988


Step,Training Loss
100,1.394500
200,0.778000
300,0.861000
400,0.675500
500,0.784700
600,0.758600
700,0.791800
800,0.658200
900,0.863300
1000,0.627200


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=29988, training_loss=0.7211353436333029, metrics={'train_runtime': 12222.4761, 'train_samples_per_second': 2.454, 'train_steps_per_second': 2.454, 'total_flos': 7.259910269553869e+16, 'train_loss': 0.7211353436333029, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to ./results
Configuration saved in ./results/config.json
Model weights saved in ./results/pytorch_model.bin
tokenizer config file saved in ./results/tokenizer_config.json
Special tokens file saved in ./results/special_tokens_map.json


'https://huggingface.co/JakeMSc/pegasus-large-legal/commit/f79d97938702120d961c709ee8449de8c21a9b53'